In [29]:
# %load decision_tree_starter.py
import numpy as np
from numpy import genfromtxt
import scipy.io
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
import pandas as pd
from sklearn.preprocessing import Imputer, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold

eps = 1e-5  # a small number


class DecisionTree:

    def __init__(self, max_depth=3, feature_labels=None):
        self.max_depth = max_depth
        self.features = feature_labels
        self.left, self.right = None, None  # for non-leaf nodes
        self.split_idx, self.thresh = None, None  # for non-leaf nodes
        self.data, self.pred = None, None  # for leaf nodes

    @staticmethod
    def entropy(y):
        c = dict(zip(*np.unique(y, return_counts=True)))
        return sum([c[i]/len(y) * np.log(1/(c[i]/len(y))) for i in c])

    @staticmethod
    def information_gain(X, y, thresh):
        dt = DecisionTree.entropy(X)
        X_l, y_l, X_h, y_h = X[y<thresh], y[y<thresh],  X[y>=thresh], y[y>=thresh]
        p_y_l, p_y_h = len(y_l)/len(y), len(y_h)/len(y)
        dt_y_l, dt_y_h = DecisionTree.entropy(X_l), DecisionTree.entropy(X_h)
        dt_xy = p_y_l * dt_y_l + p_y_h * dt_y_h
        return dt - dt_xy

    def split(self, X, y, idx, thresh):
        X0, idx0, X1, idx1 = self.split_test(X, idx=idx, thresh=thresh)
        y0, y1 = y[idx0], y[idx1]
        return X0, y0, X1, y1

    def split_test(self, X, idx, thresh):
        idx0 = np.where(X[:,idx] < thresh)[0]
        idx1 = np.where(X[:,idx] >= thresh)[0]
        X0, X1 = X[idx0, :], X[idx1, :]
        return X0, idx0, X1, idx1

    def fit(self, X, y):
        if self.max_depth > 0:
            # compute entropy gain for all single-dimension splits,
            # thresholding with a linear interpolation of 10 values
            gains = []
            thresh = np.array([np.linspace(np.min(X[:, i]) + eps,
                                           np.max(X[:, i]) - eps, num=10) for i
                               in range(X.shape[1])])
            for i in range(X.shape[1]):
                gains.append([self.information_gain(X[:, i], y, t) for t in
                              thresh[i, :]])

            gains = np.nan_to_num(np.array(gains))
            self.split_idx, thresh_idx = np.unravel_index(np.argmax(gains),
                                                          gains.shape)
            self.thresh = thresh[self.split_idx, thresh_idx]
            X0, y0, X1, y1 = self.split(X, y, idx=self.split_idx,
                                        thresh=self.thresh)
            if X0.size > 0 and X1.size > 0:
                self.left = DecisionTree(max_depth=self.max_depth-1,
                                         feature_labels=self.features)
                self.left.fit(X0, y0)
                self.right = DecisionTree(max_depth=self.max_depth-1,
                                          feature_labels=self.features)
                self.right.fit(X1, y1)
            else:
                self.max_depth = 0
                self.data, self.labels = X, y
                self.pred = stats.mode(y).mode[0]
        else:
            self.data, self.labels = X, y
            self.pred = stats.mode(y).mode[0]
        return self

    def predict(self, X):
        if self.max_depth == 0:
            return self.pred * np.ones(X.shape[0])
        else:
            X0, idx0, X1, idx1 = self.split_test(X, idx=self.split_idx,
                                                 thresh=self.thresh)
            yhat = np.zeros(X.shape[0])
            yhat[idx0] = self.left.predict(X0)
            yhat[idx1] = self.right.predict(X1)
            return yhat


class BaggedTrees(BaseEstimator, ClassifierMixin):

    def __init__(self, params=None, n=200):
        if params is None:
            params = {}
        self.params = params
        self.n = n
        self.decision_trees = [
            DecisionTreeClassifier(random_state=i, **self.params) for i in
            range(self.n)]

    def fit(self, X, y):
        for dt in self.decision_trees:
            i = np.random.choice(len(X), len(X))
            X, y = X[i], y[i]
            dt.fit(X,y)

    def predict(self, X):
        preds = [tree.predict(X) for tree in self.decision_trees]
        return stats.mode(preds)[0][0]

def split(tree, y):
    dts = tree.decision_trees
    d = {}
    for dt in dts:
        s = (y[dt.tree_.feature[0]], dt.tree_.threshold[0])
        d[s] = d.get(s, 0) + 1
    return d

class RandomForest(BaggedTrees):
    def __init__(self, params=None, n=200, m=1): 
        if params is None:
            params = {}
        params['max_features'] = m
        self.n = n
        super().__init__(params, n)
        self.m = m
        self.features = [[] for i in range(self.n)]
class BoostedRandomForest(RandomForest):
    def fit(self, X, y):
        self.w = np.ones(X.shape[0]) / X.shape[0]  # Weights on data
        self.a = np.zeros(self.n)  # Weights on decision trees
        for i in range(self.n):
            dt = self.decision_trees[i]
            datum = np.random.choice(len(X[0]), self.m, replace=False)
            self.features[i] = datum
            Xc = X.T[datum, :].T
            idx = np.random.choice(len(Xc), size=len(Xc), p=self.w/sum(self.w))
            xs, ys = Xc[idx], y[idx]
            dt.fit(xs, ys)
            pred = dt.predict(Xc)
            err = sum(self.w[pred != y]) / sum(self.w)
            self.a[i] = .5 * np.log((1 - err)/err)
            for j in range(len(self.w)):
                if (pred != y)[j]:
                    self.w[j] *= np.exp(self.a[i])
                else:
                    self.w[j] *= np.exp(- self.a[i])
        return self
    def predict(self, X):
        res = np.zeros(X.shape[0])
        for j in range(self.n):
            dt = self.decision_trees[j]
            pred = dt.predict(X[:, self.features[j]]) * self.a[j]
            res += pred
        tip = sum(self.a)/2
        res = [int(r > tip) for r in res]
        return res

def visualize(dt, level=0):
    indent = "    " * level
    if dt.pred != None:
        print(indent + str(dt.pred))
    else:
        if dt.left:
            print(indent + dt.features[dt.split_idx] + " < " + str(dt.thresh))
            visualize(dt.left, level + 1)
        if dt.right:
            print(indent + dt.features[dt.split_idx] + " > " + str(dt.thresh))
            visualize(dt.right, level + 1)

def main_method(dataset = "spam", method = "dt", crossval = False):
    params = {
        "max_depth": 5,
        # "random_state": 6,
        "min_samples_leaf": 10,
    }
    N = 100
    if dataset == "James":
        path_train, path_test = 'labels.csv', 'test.csv'
        df, dft = pd.read_csv(path_train), pd.read_csv(path_test)
        class_names = ["Right", "Left"]
        features = df.columns.values[1:]
        y = df['label']
        df = df.drop('label')
        X, Z = df, dft.values
    elif dataset == "titanic":
        path_train, path_test = 'titanic_training.csv', 'titanic_testing_data.csv'
        df, dft = pd.read_csv(path_train), pd.read_csv(path_test)
        df, dft = df.drop(['cabin', 'ticket'], axis=1), dft.drop(['cabin', 'ticket'], axis=1)
        df.sex, dft.sex = 1*(df.sex=='male'), 1*(dft.sex=='male')
        df, dft = pd.get_dummies(df), pd.get_dummies(dft)
        cols = df.columns
        features = cols.values[1:]
        df, dft = pd.DataFrame(Imputer().fit_transform(df)), pd.DataFrame(Imputer().fit_transform(df_test))
        df.columns, dft.columns = cols, cols[1:]
        y = df.survived.values
        class_names = ["Died", "Survived"]
        X, Z = df.values[:, 1:], df_test.values
        
    elif dataset == "spam":
        features = ["pain", "private", "bank", "money", "drug", "spam",
                    "prescription", "creative", "height", "featured", "differ",
                    "width", "other", "energy", "business", "message",
                    "volumes", "revision", "path", "meter", "memo", "planning",
                    "pleased", "record", "out", "semicolon", "dollar", "sharp",
                    "exclamation", "parenthesis", "square_bracket", "ampersand"]
        assert len(features) == 32

        # Load spam data
        path_train = 'spam_data.mat'
        data = scipy.io.loadmat(path_train)
        X = data['training_data']
        y = np.squeeze(data['training_labels'])
        Z = data['test_data']
        class_names = ["Ham", "Spam"]
    else:
        raise NotImplementedError("Dataset %s not handled" % dataset)

    print("Features", features)
    print("Train/test size", X.shape, Z.shape)

    print("\n\nPart 0: constant classifier")
    print("Accuracy", 1 - np.sum(y) / y.size)
    if crossval:
        for model in [DecisionTree, BaggedTrees, RandomForest, BoostedRandomForest]:
            print(model)
            kf = KFold(shuffle=True)
            for i, j in kf.split(X):
                dt = model()
                dt.fit(X[i], np.round(y[i]))
                print("Training accuracy: ", sum(dt.predict(X[i])==y[i])/len(y[i]))
                print("Testing accuracy: ", sum(dt.predict(X[j])==y[j])/len(y[j]))
    else:
        if (method == "dt"):
            # Basic decision tree
            print("\n\nPart (a-b): simplified decision tree")
            dt = DecisionTree(max_depth=3, feature_labels=features)
            dt.fit(X, y)
            visualize(dt)
        elif (method == "bagged"):
            print("\n\nPart (d-e): bagged decision tree")
            dt = BaggedTrees(n=100)
            dt.fit(X, np.round(y))
            print("Most common split: ", split(dt, y))
        elif (method == "rf"):
            print("\n\nPart (f-g): random forest")
            dt = RandomForest()
            dt.fit(X, np.round(y))
            print("Most common split: ", split(dt, y))
        elif (method == "boosted"):
            print("\n\nPart (h-i): boosted random forest")
            dt = BoostedRandomForest()
            dt.fit(X, np.round(y))
            print("Most common split: ", split(dt, y))
        pred = dt.predict(Z)
        f = open('submission.txt', 'w') 
        for val in pred:
            f.write(str(int(val)) + '\n') 
        f.close()
        print("Predictions", pred[:10])
        print("Accuracy: ", sum(dt.predict(X)==y)/len(y))
    # TODO implement and evaluate parts c-h


In [30]:
main_method("James")

Features [' pets' ' inside' ' sex' ' cup' ' morning' ' clean' ' religious' ' kids'
 ' eatin' ' sports' ' politics' ' spontaneous' ' long-term'
 ' procrastinates' ' travels' ' introvert' ' family']
Train/test size (299, 18) (100, 18)


Part 0: constant classifier
Accuracy 0.5016722408026756


Part (a-b): simplified decision tree


TypeError: unhashable type: 'slice'

In [26]:
path_train, path_test = 'labels.csv', 'test.csv'
df, dft = pd.read_csv(path_train), pd.read_csv(path_test)
class_names = ["Right", "Left"]
X, Z = df.values, dft.values

In [28]:
dft

,label,pets,inside,sex,cup,morning,clean,religious,kids,eatin,sports,politics,spontaneous,long-term,procrastinates,travels,introvert,family
1,1,1,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1
1,0,0,0,0,1,0,1,0,1,1,1,1,0,0,1,0,0,1
0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0
0,0,0,1,1,1,0,0,1,0,1,1,1,1,1,0,0,0,1
0,1,1,1,0,0,1,1,1,0,0,0,1,1,0,1,0,1,0
0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0
1,1,1,1,1,1,0,0,0,0,1,1,1,0,1,0,1,1,0
0,0,1,1,0,1,1,0,1,0,1,0,1,0,1,0,1,1,1
1,1,1,1,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1
0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,1,1,1


In [4]:
main_method("James")

NotImplementedError: Dataset James not handled

In [331]:
main_method("spam", "bagged")

Features ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'prescription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'record', 'out', 'semicolon', 'dollar', 'sharp', 'exclamation', 'parenthesis', 'square_bracket', 'ampersand']
Train/test size (5172, 32) (5857, 32)


Part 0: constant classifier
Accuracy 0.709976798144


Part (d-e): bagged decision tree
Most common split:  {(1, 0.5): 75, (1, 1.5): 25}
Predictions [0 0 0 0 0 0 0 0 0 0]
Accuracy:  0.798917246713


In [332]:
main_method("titanic", "bagged")

Features ['pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked_C' 'embarked_Q'
 'embarked_S']
Train/test size (1000, 9) (310, 9)


Part 0: constant classifier
Accuracy 0.613613613614


Part (d-e): bagged decision tree
Most common split:  {(0.0, 9.5): 33, (0.0, 7.1812500953674316): 11, (0.0, 0.5): 30, (0.0, 39.5): 6, (0.0, 15.5): 20}
Predictions [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.]
Accuracy:  0.716


In [333]:
main_method("spam", "rf")

Features ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'prescription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'record', 'out', 'semicolon', 'dollar', 'sharp', 'exclamation', 'parenthesis', 'square_bracket', 'ampersand']
Train/test size (5172, 32) (5857, 32)


Part 0: constant classifier
Accuracy 0.709976798144


Part (f-g): random forest
Most common split:  {(1, 2.5): 13, (1, 3.5): 1, (1, 22.5): 1, (1, 0.5): 162, (1, 1.5): 21, (1, 4.5): 2}
Predictions [1 1 1 0 0 0 0 0 0 1]
Accuracy:  0.750386697602


In [334]:
main_method("titanic", "rf")

Features ['pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked_C' 'embarked_Q'
 'embarked_S']
Train/test size (1000, 9) (310, 9)


Part 0: constant classifier
Accuracy 0.613613613614


Part (f-g): random forest
Most common split:  {(0.0, 1.5): 11, (0.0, 33.443748474121094): 3, (0.0, 0.5): 59, (0.0, 20.0): 1, (0.0, 10.870849609375): 2, (0.0, 19.5): 2, (0.0, 77.777099609375): 1, (0.0, 7.9875001907348633): 3, (0.0, 2.5): 22, (0.0, 17.5): 8, (0.0, 16.0): 2, (0.0, 42.5): 4, (0.0, 30.129714965820313): 3, (0.0, 9.2124996185302734): 7, (0.0, 30.629714965820313): 1, (0.0, 3.0): 9, (0.0, 28.629714965820313): 1, (0.0, 3.5): 8, (1.0, 0.5): 52, (0.0, 74.925003051757813): 1}
Predictions [ 1.  1.  0.  0.  0.  1.  1.  1.  0.  1.]
Accuracy:  0.735


In [335]:
main_method("titanic", "boosted")

Features ['pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked_C' 'embarked_Q'
 'embarked_S']
Train/test size (1000, 9) (310, 9)


Part 0: constant classifier
Accuracy 0.613613613614


Part (h-i): boosted random forest
Most common split:  {(0.0, 8.3833503723144531): 1, (0.0, 0.2522522509098053): 1, (0.0, 53.5): 1, (0.0, 20.5): 1, (0.0, 14.5): 1, (0.0, 56.197898864746094): 1, (0.0, 82.21875): 1, (0.0, 7.8833498954772949): 1, (0.0, 50.987499237060547): 1, (0.0, 0.95835000276565552): 1, (0.0, 10.0): 1, (0.0, 262.6875): 2, (0.0, 3.5): 12, (0.0, 3.0): 1, (0.0, 28.450000762939453): 1, (0.0, 32.25): 1, (0.0, 15.5): 1, (0.0, 6.6999998092651367): 1, (0.0, 50.931251525878906): 1, (0.0, 44.595848083496094): 1, (0.0, 4.5): 8, (0.0, 62.5): 1, (0.0, 7.8667001724243164): 1, (0.0, 0.5): 91, (0.0, 51.931251525878906): 1, (0.0, 9.2749996185302734): 1, (0.0, 5.5): 1, (0.0, 7.9875001907348633): 1, (0.0, 8.7562503814697266): 1, (0.0, 158.20834350585938): 1, (0.0, 8.0396003723144531): 1, (0.0, 1.5): 22, (0

In [346]:
main_method("spam", "boosted")

Features ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'prescription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'record', 'out', 'semicolon', 'dollar', 'sharp', 'exclamation', 'parenthesis', 'square_bracket', 'ampersand']
Train/test size (5172, 32) (5857, 32)


Part 0: constant classifier
Accuracy 0.709976798144


Part (h-i): boosted random forest
Most common split:  {(1, 2.5): 13, (1, 34.5): 1, (1, 3.5): 10, (1, 22.5): 1, (1, 12.5): 2, (1, 5.5): 9, (1, 26.0): 1, (1, 8.5): 3, (1, 4.5): 1, (1, 37.0): 3, (1, 8.0): 1, (1, 6.0): 1, (1, 7.5): 1, (1, 6.5): 2, (1, 0.5): 127, (1, 30.0): 1, (1, 1.5): 23}
Predictions [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy:  0.818832173241


In [345]:
main_method("spam", crossval=True)

Features ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'prescription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'record', 'out', 'semicolon', 'dollar', 'sharp', 'exclamation', 'parenthesis', 'square_bracket', 'ampersand']
Train/test size (5172, 32) (5857, 32)


Part 0: constant classifier
Accuracy 0.709976798144
<class '__main__.DecisionTree'>
Training accuracy:  0.801044083527
Testing accuracy:  0.782482598608
Training accuracy:  0.790023201856
Testing accuracy:  0.804524361949
Training accuracy:  0.793503480278
Testing accuracy:  0.797563805104
<class '__main__.BaggedTrees'>
Training accuracy:  0.777262180974
Testing accuracy:  0.769721577726
Training accuracy:  0.716647331787
Testing accuracy:  0.687354988399
Training accuracy:  0.703596287703
Testing accuracy:  0.686774941995
<class '__main__.RandomForest'>
Training accuracy:  0.680974477958
Testing ac

In [344]:
main_method("titanic", crossval=True)

Features ['pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked_C' 'embarked_Q'
 'embarked_S']
Train/test size (1000, 9) (310, 9)


Part 0: constant classifier
Accuracy 0.613613613614
<class '__main__.DecisionTree'>
Training accuracy:  0.788288288288
Testing accuracy:  0.787425149701
Training accuracy:  0.779610194903
Testing accuracy:  0.807807807808
Training accuracy:  0.799100449775
Testing accuracy:  0.768768768769
<class '__main__.BaggedTrees'>
Training accuracy:  0.779279279279
Testing accuracy:  0.805389221557
Training accuracy:  0.566716641679
Testing accuracy:  0.603603603604
Training accuracy:  0.692653673163
Testing accuracy:  0.720720720721
<class '__main__.RandomForest'>
Training accuracy:  0.698198198198
Testing accuracy:  0.595808383234
Training accuracy:  0.71964017991
Testing accuracy:  0.732732732733
Training accuracy:  0.580209895052
Testing accuracy:  0.621621621622
<class '__main__.BoostedRandomForest'>
Training accuracy:  0.906906906907
Testing accuracy:  0.739520

### part h
The trees are being weighted by whether the feature they are basing their prediction off has actually ended up being relevant in the prediction of the class. When ai<0, we should actually consider this *negatively* and the algorithm handles this by summing up over many ai's.
### part i
Really challenging data to classify is determined by looking at the most common splits; features which the data splits on least often is pretty hard to classify, because it could really go either way. Therefore, if the data doesn't contain the things which the tree splits on most commonly, things will go south. If I were the classifier, I would determine which data is easiest to classify by seeing whether it contains a lot of the symbols that I have previously made major splits on like "pain" or "bank". It would be hard if it didn't have any of these "give-aways" basically.
### part j
It looks like the best model for spam was the Boosted Random Forest, which got up to 83% validation accuracy. The Boosted Random Forest appeared very good for the titanic dataset, getting up to 90% training accuracy, but in fact the model with the best test accuracy was the simple decision tree. 
### Problem 5: Your Own Question
Prompt: Explain the motivation and intuition behind using decision trees, and tie it to a real-world example. 


Answer: A decision tree is a rule-based way of doing ML, it's a series of logisms that we learn. We need sparsity because we want to collect as many features as possible and have our model do the work of determining which ones it needs. We find the correlation between the label and the feature, but if the data is imbalanced it's hard to classify. Instead, we use entropy and mutual information; this tells us how much 'surprise' we have left -because we are gaining some number of bits of information upon seeing something new. When p=1/2 we are completely uncertain, just randomly guessing. Limit as p approaches 1 means most of the random experiments came out as expected, with just a few contradicting our expectations. What's a real-world example of this? Well, it's hard to compress when we're uncertain, so we should use...Huffman Encoding! This is precisely what we learned in CS61B and CS170; we want to use the Huffman Encoding Tree because the most 'uncertain' things are given the most bits.

In [19]:
ans = [[1,0,1],[0,0,0]]
alphabet = [['a','b'],['c','d'],['e','f']]
print([[alphabet[i][ans[j][i]] for i in range(3)] for j in range(2)])

[['b', 'c', 'f'], ['a', 'c', 'e']]


In [18]:
len(alphabet)

3

In [25]:
arr = [1]


In [26]:
arr.append(3)

In [27]:
arr

[1, 3]

In [28]:
auth = {'wellford': 1234}

In [29]:
auth['wellford']

1234

In [21]:
labels = range(4)
profiles = [5,6,7,8]

In [33]:
labels[:2]

range(0, 2)

In [34]:
labels[3:]

range(3, 4)